In [ ]:
# Downloading packages and importing
import nltk
nltk.download('stopwords')
import pandas as pd
# Import and load model
import spacy
nlp=spacy.load("en_core_web_sm")
nlp

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Error loading stop: Package 'stop' not found in index
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import io
from google.colab import files
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['train (1).csv']))

Saving train (1).csv to train (1).csv


In [ ]:
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [ ]:
df = df.drop(['id',"keyword","location","target"],1)
dfcopy=df
df

,text
0,Our Deeds are the Reason of this #earthquake M...
1,Forest fire near La Ronge Sask. Canada
2,All residents asked to 'shelter in place' are ...
3,"13,000 people receive #wildfires evacuation or..."
4,Just got sent this photo from Ruby #Alaska as ...
...,...
7608,Two giant cranes holding a bridge collapse int...
7609,@aria_ahrary @TheTawniest The out of control w...
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611,Police investigating after an e-bike collided ...


In [ ]:
'''Week 1 objectives
1) Clean the dataset as in remove stopwords, punctuation, twitter tags and urls.
2) Create inverted index for the remaining words
'''

# Removing url's from the tweets like http:\\.......
import re

for i in range(len(df["text"])):
  text=re.sub(r"http\S+", "", dfcopy["text"][i])
  dfcopy["text"][i]=text
dfcopy

,text
0,Our Deeds are the Reason of this #earthquake M...
1,Forest fire near La Ronge Sask. Canada
2,All residents asked to 'shelter in place' are ...
3,"13,000 people receive #wildfires evacuation or..."
4,Just got sent this photo from Ruby #Alaska as ...
...,...
7608,Two giant cranes holding a bridge collapse int...
7609,@aria_ahrary @TheTawniest The out of control w...
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii.
7611,Police investigating after an e-bike collided ...


In [ ]:
# Removing contractions from the tweets for dimensionality reduction
# Contractions are short form of writing words like I'm for I am or shouldn't for should not
import sys  
!{sys.executable} -m pip install contractions
import contractions

for i in range(len(df["text"])):
  text=contractions.fix(dfcopy["text"][i])
  dfcopy["text"][i]=text
dfcopy

     |████████████████████████████████| 284 kB 13.2 MB/s 
     |████████████████████████████████| 321 kB 68.2 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85449 sha256=025a0f89b6e63cddeff75bc2af19f314cd487f8bd2bf2f60afbb6acc48bb8ffb
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


,text
0,Our Deeds are the Reason of this #earthquake M...
1,Forest fire near La Ronge Sask. Canada
2,All residents asked to 'shelter in place' are ...
3,"13,000 people receive #wildfires evacuation or..."
4,Just got sent this photo from Ruby #Alaska as ...
...,...
7608,Two giant cranes holding a bridge collapse int...
7609,@aria_ahrary @TheTawniest The out of control w...
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii.
7611,Police investigating after an e-bike collided ...


In [ ]:
# Tokenising at first glance

tok=[]
for i in dfcopy["text"]:
  tok.append(nltk.word_tokenize(i))
tok

[['Our',
  'Deeds',
  'are',
  'the',
  'Reason',
  'of',
  'this',
  '#',
  'earthquake',
  'May',
  'ALLAH',
  'Forgive',
  'us',
  'all'],
 ['Forest', 'fire', 'near', 'La', 'Ronge', 'Sask', '.', 'Canada'],
 ['All',
  'residents',
  'asked',
  'to',
  "'shelter",
  'in',
  'place',
  "'",
  'are',
  'being',
  'notified',
  'by',
  'officers',
  '.',
  'No',
  'other',
  'evacuation',
  'or',
  'shelter',
  'in',
  'place',
  'orders',
  'are',
  'expected'],
 ['13,000',
  'people',
  'receive',
  '#',
  'wildfires',
  'evacuation',
  'orders',
  'in',
  'California'],
 ['Just',
  'got',
  'sent',
  'this',
  'photo',
  'from',
  'Ruby',
  '#',
  'Alaska',
  'as',
  'smoke',
  'from',
  '#',
  'wildfires',
  'pours',
  'into',
  'a',
  'school'],
 ['#',
  'RockyFire',
  'Update',
  '=',
  '>',
  'California',
  'Hwy',
  '.',
  '20',
  'closed',
  'in',
  'both',
  'directions',
  'due',
  'to',
  'Lake',
  'County',
  'fire',
  '-',
  '#',
  'CAfire',
  '#',
  'wildfires'],
 ['#',
  

In [ ]:
# Converting tokens to lower case

new_words = []
for i in tok:
  text=[]
  for j in i:
    new_word = j.lower()
    text.append(new_word)
  new_words.append(text)
new_words
    

[['our',
  'deeds',
  'are',
  'the',
  'reason',
  'of',
  'this',
  '#',
  'earthquake',
  'may',
  'allah',
  'forgive',
  'us',
  'all'],
 ['forest', 'fire', 'near', 'la', 'ronge', 'sask', '.', 'canada'],
 ['all',
  'residents',
  'asked',
  'to',
  "'shelter",
  'in',
  'place',
  "'",
  'are',
  'being',
  'notified',
  'by',
  'officers',
  '.',
  'no',
  'other',
  'evacuation',
  'or',
  'shelter',
  'in',
  'place',
  'orders',
  'are',
  'expected'],
 ['13,000',
  'people',
  'receive',
  '#',
  'wildfires',
  'evacuation',
  'orders',
  'in',
  'california'],
 ['just',
  'got',
  'sent',
  'this',
  'photo',
  'from',
  'ruby',
  '#',
  'alaska',
  'as',
  'smoke',
  'from',
  '#',
  'wildfires',
  'pours',
  'into',
  'a',
  'school'],
 ['#',
  'rockyfire',
  'update',
  '=',
  '>',
  'california',
  'hwy',
  '.',
  '20',
  'closed',
  'in',
  'both',
  'directions',
  'due',
  'to',
  'lake',
  'county',
  'fire',
  '-',
  '#',
  'cafire',
  '#',
  'wildfires'],
 ['#',
  

In [ ]:
# Removing punctuations from the tokens

unpunc = []
for i in new_words:
  text=[]
  for j in i:
    new_word = re.sub(r'[^\w\s]', '', j)
    if new_word != '':
      text.append(new_word)
  unpunc.append(text)
unpunc  

[['our',
  'deeds',
  'are',
  'the',
  'reason',
  'of',
  'this',
  'earthquake',
  'may',
  'allah',
  'forgive',
  'us',
  'all'],
 ['forest', 'fire', 'near', 'la', 'ronge', 'sask', 'canada'],
 ['all',
  'residents',
  'asked',
  'to',
  'shelter',
  'in',
  'place',
  'are',
  'being',
  'notified',
  'by',
  'officers',
  'no',
  'other',
  'evacuation',
  'or',
  'shelter',
  'in',
  'place',
  'orders',
  'are',
  'expected'],
 ['13000',
  'people',
  'receive',
  'wildfires',
  'evacuation',
  'orders',
  'in',
  'california'],
 ['just',
  'got',
  'sent',
  'this',
  'photo',
  'from',
  'ruby',
  'alaska',
  'as',
  'smoke',
  'from',
  'wildfires',
  'pours',
  'into',
  'a',
  'school'],
 ['rockyfire',
  'update',
  'california',
  'hwy',
  '20',
  'closed',
  'in',
  'both',
  'directions',
  'due',
  'to',
  'lake',
  'county',
  'fire',
  'cafire',
  'wildfires'],
 ['flood',
  'disaster',
  'heavy',
  'rain',
  'causes',
  'flash',
  'flooding',
  'of',
  'streets',
  '

In [ ]:
# COnverting numbers to words
import inflect

p = inflect.engine()
numb = []
for i in unpunc:
  text=[]
  for j in i:
    if j.isdigit():
      j = p.number_to_words(j)  
    text.append(j)
  numb.append(text)
numb  
  

[['our',
  'deeds',
  'are',
  'the',
  'reason',
  'of',
  'this',
  'earthquake',
  'may',
  'allah',
  'forgive',
  'us',
  'all'],
 ['forest', 'fire', 'near', 'la', 'ronge', 'sask', 'canada'],
 ['all',
  'residents',
  'asked',
  'to',
  'shelter',
  'in',
  'place',
  'are',
  'being',
  'notified',
  'by',
  'officers',
  'no',
  'other',
  'evacuation',
  'or',
  'shelter',
  'in',
  'place',
  'orders',
  'are',
  'expected'],
 ['thirteen thousand',
  'people',
  'receive',
  'wildfires',
  'evacuation',
  'orders',
  'in',
  'california'],
 ['just',
  'got',
  'sent',
  'this',
  'photo',
  'from',
  'ruby',
  'alaska',
  'as',
  'smoke',
  'from',
  'wildfires',
  'pours',
  'into',
  'a',
  'school'],
 ['rockyfire',
  'update',
  'california',
  'hwy',
  'twenty',
  'closed',
  'in',
  'both',
  'directions',
  'due',
  'to',
  'lake',
  'county',
  'fire',
  'cafire',
  'wildfires'],
 ['flood',
  'disaster',
  'heavy',
  'rain',
  'causes',
  'flash',
  'flooding',
  'of',


In [ ]:
# Removing stopwords in nltk
from nltk.corpus import stopwords
my_stopwords=set(stopwords.words('english'))

stop = []
for i in numb:
  text=[]
  for j in i:
    if j not in my_stopwords:
      text.append(j)
  stop.append(text)
stop
  

[['deeds', 'reason', 'earthquake', 'may', 'allah', 'forgive', 'us'],
 ['forest', 'fire', 'near', 'la', 'ronge', 'sask', 'canada'],
 ['residents',
  'asked',
  'shelter',
  'place',
  'notified',
  'officers',
  'evacuation',
  'shelter',
  'place',
  'orders',
  'expected'],
 ['thirteen thousand',
  'people',
  'receive',
  'wildfires',
  'evacuation',
  'orders',
  'california'],
 ['got',
  'sent',
  'photo',
  'ruby',
  'alaska',
  'smoke',
  'wildfires',
  'pours',
  'school'],
 ['rockyfire',
  'update',
  'california',
  'hwy',
  'twenty',
  'closed',
  'directions',
  'due',
  'lake',
  'county',
  'fire',
  'cafire',
  'wildfires'],
 ['flood',
  'disaster',
  'heavy',
  'rain',
  'causes',
  'flash',
  'flooding',
  'streets',
  'manitou',
  'colorado',
  'springs',
  'areas'],
 ['top', 'hill', 'see', 'fire', 'woods'],
 ['emergency', 'evacuation', 'happening', 'building', 'across', 'street'],
 ['afraid', 'tornado', 'coming', 'area'],
 ['three', 'people', 'died', 'heat', 'wave', '

In [ ]:
# Building the inverted index

index={}

documents=100
for i in range(documents):
  for j in stop[i]:
    text=[]
    if j not in index:
      for k in range(documents):
        if j in stop[k]:
          text.append(k+1)
      index[j]=text.copy()
index

{'17th': [92],
 '2k13': [63],
 '8m': [69],
 'aashiqui': [94],
 'aba': [34],
 'abia': [38],
 'ablaze': [32,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  42,
  43,
  44,
  45,
  47,
  48,
  51,
  52,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67],
 'able': [82],
 'accident': [68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100],
 'accidents': [72],
 'acquisitions': [37],
 'across': [9],
 'actress': [94],
 'advice': [71],
 'afraid': [10],
 'africanbaze': [34],
 'aggarwal': [94],
 'alaska': [5],
 'alberta': [95],
 'alexis_sanchez': [55],
 'alive': [42],
 'allah': [1],
 'alone': [47, 60],
 'alternate': [96],
 'always': [33],
 'america': [70],
 'among': [72],
 'amp': [63, 65, 68, 71],
 'ancop': [43],
 'angeles': [59],
 'another': [83],
 'anu': [94],
 'area': [10],
 'areas': [7],
 'around': [37, 82],
 'arrived': [14],
 'arsonist': [5

In [ ]:
# Lemmatizing the tokens using wordnet

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer    
lemmatizer = WordNetLemmatizer()

lemmas = []
for i in stop:
  text=[]
  for j in i:
    lemma = lemmatizer.lemmatize(j)
    text.append(lemma)
  lemmas.append(text)
lemmas

#notice how us changes to u in the first sentence

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


[['deed', 'reason', 'earthquake', 'may', 'allah', 'forgive', 'u'],
 ['forest', 'fire', 'near', 'la', 'ronge', 'sask', 'canada'],
 ['resident',
  'asked',
  'shelter',
  'place',
  'notified',
  'officer',
  'evacuation',
  'shelter',
  'place',
  'order',
  'expected'],
 ['thirteen thousand',
  'people',
  'receive',
  'wildfire',
  'evacuation',
  'order',
  'california'],
 ['got',
  'sent',
  'photo',
  'ruby',
  'alaska',
  'smoke',
  'wildfire',
  'pours',
  'school'],
 ['rockyfire',
  'update',
  'california',
  'hwy',
  'twenty',
  'closed',
  'direction',
  'due',
  'lake',
  'county',
  'fire',
  'cafire',
  'wildfire'],
 ['flood',
  'disaster',
  'heavy',
  'rain',
  'cause',
  'flash',
  'flooding',
  'street',
  'manitou',
  'colorado',
  'spring',
  'area'],
 ['top', 'hill', 'see', 'fire', 'wood'],
 ['emergency', 'evacuation', 'happening', 'building', 'across', 'street'],
 ['afraid', 'tornado', 'coming', 'area'],
 ['three', 'people', 'died', 'heat', 'wave', 'far'],
 ['haha'

In [ ]:
# Stemming with nltk's PorterStemmer

from nltk.stem import PorterStemmer 
stemmer=PorterStemmer()

stemmed_tokens=[]
for i in lemmas:
  text=[]
  for j in i:
      text.append(stemmer.stem(j))
  stemmed_tokens.append(text)
stemmed_tokens

#notice how e is removed from many words

[['deed', 'reason', 'earthquak', 'may', 'allah', 'forgiv', 'u'],
 ['forest', 'fire', 'near', 'la', 'rong', 'sask', 'canada'],
 ['resid',
  'ask',
  'shelter',
  'place',
  'notifi',
  'offic',
  'evacu',
  'shelter',
  'place',
  'order',
  'expect'],
 ['thirteen thousand',
  'peopl',
  'receiv',
  'wildfir',
  'evacu',
  'order',
  'california'],
 ['got',
  'sent',
  'photo',
  'rubi',
  'alaska',
  'smoke',
  'wildfir',
  'pour',
  'school'],
 ['rockyfir',
  'updat',
  'california',
  'hwi',
  'twenti',
  'close',
  'direct',
  'due',
  'lake',
  'counti',
  'fire',
  'cafir',
  'wildfir'],
 ['flood',
  'disast',
  'heavi',
  'rain',
  'caus',
  'flash',
  'flood',
  'street',
  'manit',
  'colorado',
  'spring',
  'area'],
 ['top', 'hill', 'see', 'fire', 'wood'],
 ['emerg', 'evacu', 'happen', 'build', 'across', 'street'],
 ['afraid', 'tornado', 'come', 'area'],
 ['three', 'peopl', 'die', 'heat', 'wave', 'far'],
 ['haha',
  'south',
  'tampa',
  'get',
  'flood',
  'hah',
  'wait',
 

In [ ]:
# Building the final inverted index

final={}

documents=100
for i in range(documents):
  for j in stemmed_tokens[i]:
    text=[]
    if j not in final:
      for k in range(documents):
        if j in stemmed_tokens[k]:
          text.append(k+1)
      final[j]=text.copy()
final

{'17th': [92],
 '2k13': [63],
 '8m': [69],
 'aashiqui': [94],
 'aba': [34],
 'abia': [38],
 'abl': [82],
 'ablaz': [32,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  42,
  43,
  44,
  45,
  47,
  48,
  51,
  52,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67],
 'accid': [68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100],
 'acquisit': [37],
 'across': [9],
 'actress': [94],
 'advic': [71],
 'afraid': [10],
 'africanbaz': [34],
 'aggarw': [94],
 'alaska': [5],
 'alberta': [95],
 'alexis_sanchez': [55],
 'aliv': [42],
 'allah': [1],
 'alon': [47, 60],
 'altern': [96],
 'alway': [33],
 'america': [70],
 'among': [72],
 'amp': [63, 65, 68, 71],
 'ancop': [43],
 'angel': [59],
 'anoth': [83],
 'anu': [94],
 'area': [7, 10],
 'around': [37, 82],
 'arriv': [14],
 'arsonist': [54],
 'ashvil': [85],
 'ask': [3],
 'av': [78],
 'ave': [5